## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [2]:
# %pip install pandas numpy scikit-learn matplotlib pycaret lightgbm xgboost catboost

In [3]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [4]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_47.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_47.csv')

In [5]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_47.csv')

# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']


### 인코딩 

In [7]:
Total_categorical_columns = [
    "시술_당시_나이",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [8]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [9]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from pycaret.classification import *

In [11]:
# 필요한 라이브러리 임포트
from pycaret.classification import *
import pandas as pd

# 특성 이름의 공백을 밑줄로 대체
Total_X.columns = Total_X.columns.str.replace(' ', '_')
Total_test.columns = Total_test.columns.str.replace(' ', '_')

# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X,
                                                                            Total_y,
                                                                            test_size=0.2,
                                                                            random_state=42,
                                                                            stratify=Total_y)

# PyCaret 설정
clf = setup(data=pd.concat([Total_X_train, Total_y_train], axis=1), 
            target='임신_성공_여부', 
            session_id=42, 
            fix_imbalance=True, 
            normalize=True, 
            feature_selection=True)

# 모델 비교 및 최상의 모델 선택
best_model = compare_models(n_select=1, sort='AUC')

[LightGBM] [Info] Number of positive: 106465, number of negative: 106465
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16225
[LightGBM] [Info] Number of data points in the train set: 212930, number of used features: 91
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,42
1,Target,임신_성공_여부
2,Target type,Binary
3,Original data shape,"(205075, 95)"
4,Transformed data shape,"(274453, 19)"
5,Transformed train set shape,"(212930, 19)"
6,Transformed test set shape,"(61523, 19)"
7,Numeric features,94
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7403,0.7332,0.2284,0.4945,0.3124,0.1782,0.1985,5.2550
gbc,Gradient Boosting Classifier,0.7228,0.7303,0.3856,0.4567,0.4182,0.2379,0.2394,11.4800
catboost,CatBoost Classifier,0.7434,0.7298,0.1827,0.5094,0.2689,0.1534,0.1832,24.2960
xgboost,Extreme Gradient Boosting,0.7407,0.7289,0.2055,0.4960,0.2906,0.1640,0.1879,5.0380
ada,Ada Boost Classifier,0.6788,0.7193,0.5694,0.4121,0.4780,0.2546,0.2617,6.0270
ridge,Ridge Classifier,0.6117,0.7170,0.7521,0.3747,0.5002,0.2370,0.2758,3.7790
lda,Linear Discriminant Analysis,0.6117,0.7170,0.7522,0.3747,0.5002,0.2371,0.2759,4.1330
lr,Logistic Regression,0.6167,0.7145,0.7361,0.3764,0.4981,0.2373,0.2725,4.1140
svm,SVM - Linear Kernel,0.5984,0.7141,0.7752,0.3684,0.4993,0.2294,0.2744,4.0530
qda,Quadratic Discriminant Analysis,0.5681,0.6940,0.7928,0.3512,0.4868,0.2005,0.2509,3.9960


In [13]:
best_model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# 최상의 모델 하이퍼파라미터 튜닝
tuned_model = tune_model(best_model, optimize='AUC')

# 최상의 모델 학습
final_model = finalize_model(tuned_model)

# 테스트 데이터 예측
y_pred = predict_model(final_model, data=Total_X_test)

# 평가 지표 계산
accuracy = accuracy_score(Total_y_test, y_pred['Label'])
f1 = f1_score(Total_y_test, y_pred['Label'])
auc = roc_auc_score(Total_y_test, y_pred['Score'])
cm = confusion_matrix(Total_y_test, y_pred['Label'])

# 결과 출력
print("--- Model Performance ---")
print(f"Model Accuracy: {accuracy}")
print(f"Model F1 Score: {f1}")
print(f"Model AUC: {auc}")

# 혼동 행렬 출력
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=final_model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# %pip install optuna

In [ ]:
# # 데이터 로드 및 전처리 (X_train, X_valid, y_train, y_valid)
# from sklearn.linear_model import RidgeClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# import optuna

# def objective(trial):
#     alpha = trial.suggest_float('alpha', 0.1, 10.0)
#     solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])

#     model = RidgeClassifier(
#         alpha=alpha,
#         solver=solver,
#         random_state=42
#     )

#     model.fit(X_train, y_train)
#     preds = model.predict(X_valid)
#     score = accuracy_score(y_valid, preds)
#     return score

# X_train, X_valid, y_train, y_valid = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42, stratify=Total_y)

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=300)

# print('Best trial:', study.best_trial.params)
# print('Best score:', study.best_value)

[I 2025-02-22 00:21:55,170] A new study created in memory with name: no-name-8b33a344-9698-4688-ae0f-7a29c8864bdb
[I 2025-02-22 00:21:55,494] Trial 0 finished with value: 0.7422223955996802 and parameters: {'alpha': 8.342488470720877, 'solver': 'auto'}. Best is trial 0 with value: 0.7422223955996802.
[I 2025-02-22 00:21:58,320] Trial 1 finished with value: 0.7422223955996802 and parameters: {'alpha': 9.686527009154059, 'solver': 'svd'}. Best is trial 0 with value: 0.7422223955996802.


In [10]:
# 필요한 라이브러리 임포트
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split

# RidgeClassifier 모델 정의
Total_model = RidgeClassifier(random_state=42)

# 모든 학습 데이터를 사용하여 모델 학습
Total_model.fit(Total_X, Total_y)

# 테스트 데이터 예측
Total_pred_scores = Total_model.decision_function(Total_test.drop('ID', axis=1))

# 예측 점수를 테스트 데이터에 추가
Total_test['probability'] = Total_pred_scores

# 최종 제출 파일 생성
submission = Total_test[['ID', 'probability']]
submission = submission.sort_values(by='ID')

# 제출 파일 저장
submission.to_csv('../submission/code47_all_ridge.csv', index=False, encoding='utf-8')

.